In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install -U transformers datasets accelerate trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 98.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninst

In [3]:
pip uninstall -y trl

Found existing installation: trl 0.17.0
Uninstalling trl-0.17.0:
  Successfully uninstalled trl-0.17.0
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install trl==0.11.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 8.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
import trl
import transformers
import accelerate
import torch

print(f"Phiên bản trl: {trl.__version__}")
print(f"Phiên bản transformers: {transformers.__version__}")
print(f"Phiên bản accelerate: {accelerate.__version__}")
print(f"Phiên bản torch: {torch.__version__}")

Phiên bản trl: 0.11.3
Phiên bản transformers: 4.51.3
Phiên bản accelerate: 1.6.0
Phiên bản torch: 2.5.1+cu124


In [6]:
from trl import PPOConfig
import inspect # Thư viện để kiểm tra code

try:
    # Cách 1: In ra tài liệu hướng dẫn (docstring) của hàm khởi tạo
    # print("--- Help on PPOConfig ---")
    # help(PPOConfig)

    # Cách 2: Lấy chữ ký (signature) của hàm khởi tạo để xem các tham số hợp lệ
    print("\n--- Signature of PPOConfig.__init__ ---")
    print(inspect.signature(PPOConfig.__init__))

except Exception as e:
    print(f"Không thể kiểm tra PPOConfig: {e}")
    print("Hãy chắc chắn rằng thư viện trl đã được cài đặt đúng.")

2025-04-27 09:48:00.595330: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745747280.786337      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745747280.840790      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered



--- Signature of PPOConfig.__init__ ---
(self, exp_name: str = 'colab_kernel_launcher', seed: int = 0, log_with: Optional[Literal['wandb', 'tensorboard']] = None, task_name: Optional[str] = None, model_name: str = 'gpt2', query_dataset: str = 'stanfordnlp/imdb', reward_model: str = 'sentiment-analysis:lvwerra/distilbert-imdb', remove_unused_columns: bool = True, tracker_kwargs: Annotated[Optional[dict], _ArgConfig(name=None, metavar='JSON', help=None, help_behavior_hint=None, aliases=None, prefix_name=None, constructor_factory=<function arg.<locals>.<lambda> at 0x7b906ae08d60>, default=<NonpropagatingMissingType id='135860201607760'>)] = <factory>, accelerator_kwargs: Annotated[Optional[dict], _ArgConfig(name=None, metavar='JSON', help=None, help_behavior_hint=None, aliases=None, prefix_name=None, constructor_factory=<function arg.<locals>.<lambda> at 0x7b906ae08d60>, default=<NonpropagatingMissingType id='135860201607760'>)] = <factory>, project_kwargs: Annotated[Optional[dict], _Arg

In [7]:
# -*- coding: utf-8 -*-
import torch
import warnings
import trl
from datasets import load_dataset, Dataset # Giữ lại import này
from torch.utils.data import Dataset as TorchDataset # Import PyTorch Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    pipeline,
    BitsAndBytesConfig # Tùy chọn nếu cần quantization
)
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
from trl.core import LengthSampler
import accelerate # Cần thiết cho PPOTrainer
import numpy as np
import os

# --- Cấu hình Chung ---
base_llm_model_name = "distilgpt2"  # LLM nhỏ để tinh chỉnh PPO
reward_model_name = "distilbert-base-uncased" # Model cho RM (phân loại cảm xúc)
dataset_name_rm = "imdb" # Dataset để huấn luyện RM
output_dir_rm = "./sentiment_reward_model"
output_dir_ppo = "./ppo_positive_generator"
device = "cuda" if torch.cuda.is_available() else "cpu" # Kiểm tra GPU

print(f"Sử dụng thiết bị: {device}")
if device == "cpu":
    warnings.warn("Đang chạy trên CPU. Quá trình huấn luyện sẽ rất chậm.")

# --- Bước 1: Huấn luyện Reward Model (Sentiment Classifier) ---

print("\n--- Bắt đầu Bước 1: Huấn luyện Reward Model (Sentiment Classifier) ---")

# 1.1. Load Tokenizer và Model cho RM
rm_tokenizer = AutoTokenizer.from_pretrained(reward_model_name)
rm_model = AutoModelForSequenceClassification.from_pretrained(reward_model_name, num_labels=2) # 2 nhãn: negative (0), positive (1)
rm_model.to(device) # Chuyển model lên device

# 1.2. Load và Chuẩn bị Dữ liệu IMDB
def preprocess_function_rm(examples):
    # Tokenize text cho mô hình phân loại
    return rm_tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

try:
    # Load một phần nhỏ dataset để demo
    imdb_dataset = load_dataset(dataset_name_rm, split="train[:1000]+test[:200]") # 1000 train, 200 test
    imdb_dataset = imdb_dataset.map(preprocess_function_rm, batched=True)
    imdb_dataset = imdb_dataset.rename_column("label", "labels") # Đổi tên cột label để phù hợp với Trainer
    imdb_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

    # Chia thành tập train và eval một cách tường minh
    train_dataset_rm = imdb_dataset.filter(lambda example, idx: idx < 1000, with_indices=True)
    eval_dataset_rm = imdb_dataset.filter(lambda example, idx: idx >= 1000, with_indices=True)
    print(f"Đã load {len(train_dataset_rm)} mẫu train và {len(eval_dataset_rm)} mẫu eval cho RM.")

except Exception as e:
    print(f"Lỗi khi load/xử lý dataset {dataset_name_rm}: {e}")
    print("Huấn luyện RM bị hủy.")
    exit()

# 1.3. Cấu hình Training Arguments cho RM
training_args_rm = TrainingArguments(
    output_dir=os.path.join(output_dir_rm, "trainer_checkpoints"), # Lưu checkpoint vào thư mục con
    num_train_epochs=1, # Chỉ huấn luyện 1 epoch cho demo
    per_device_train_batch_size=8, # Giảm nếu OOM
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    learning_rate=2e-5,
    eval_strategy="epoch", # Đánh giá cuối mỗi epoch
    save_strategy="epoch", # Lưu cuối mỗi epoch
    logging_steps=50,
    load_best_model_at_end=True, # Load model tốt nhất dựa trên eval loss
    metric_for_best_model="loss", # Sử dụng loss để xác định model tốt nhất
    greater_is_better=False,     # Loss thấp hơn là tốt hơn
    report_to="none", # Tắt logging bên ngoài (wandb/tensorboard)
)

# 1.4. Khởi tạo Trainer cho RM
rm_trainer = Trainer(
    model=rm_model,
    args=training_args_rm,
    train_dataset=train_dataset_rm,
    eval_dataset=eval_dataset_rm,
    tokenizer=rm_tokenizer,
)

# 1.5. Huấn luyện RM
print("Huấn luyện RM...")
rm_trainer.train()

# 1.6. Đánh giá RM
print("Đánh giá RM...")
eval_results = rm_trainer.evaluate()
print(f"Kết quả đánh giá RM: {eval_results}")

# 1.7. Lưu RM tốt nhất
rm_trainer.save_model(output_dir_rm)
rm_tokenizer.save_pretrained(output_dir_rm)
print(f"Reward Model (Sentiment Classifier) đã được lưu tại: {output_dir_rm}")

# Giải phóng bộ nhớ RM (nếu cần)
del rm_model, rm_trainer, imdb_dataset, train_dataset_rm, eval_dataset_rm
if device == "cuda":
    torch.cuda.empty_cache()

# --- Bước 2: Fine-tuning bằng PPO để tạo văn bản Tích cực ---

print("\n--- Bắt đầu Bước 2: Fine-tuning bằng PPO ---")

# 2.1. Load Tokenizer và Models cho PPO
ppo_tokenizer = AutoTokenizer.from_pretrained(base_llm_model_name)
if ppo_tokenizer.pad_token is None:
    ppo_tokenizer.pad_token = ppo_tokenizer.eos_token # Cần pad token

# Load Reward Model đã huấn luyện (chỉ cần forward pass)
reward_tokenizer = AutoTokenizer.from_pretrained(output_dir_rm)
# Load model để đánh giá, đảm bảo nó ở đúng device
reward_model = AutoModelForSequenceClassification.from_pretrained(output_dir_rm)
reward_model.to(device) # Chuyển RM lên device
reward_model.eval() # Chuyển sang chế độ đánh giá

# Load LLM cơ sở với Value Head cho PPO
# device_map="auto" sẽ tự động phân phối lên các GPU nếu có
# Thêm quantization nếu cần và có thư viện bitsandbytes
# bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.bfloat16)
ppo_model = AutoModelForCausalLMWithValueHead.from_pretrained(
    base_llm_model_name,
    # quantization_config=bnb_config, # Bỏ comment nếu dùng 4-bit
    pad_token_id=ppo_tokenizer.pad_token_id,
    device_map={"": device} if device == "cpu" else "auto", # Chỉ định device map cho CPU hoặc để accelerate xử lý GPU
    # torch_dtype=torch.bfloat16 # Nếu dùng GPU hỗ trợ
)

# Model tham chiếu (không huấn luyện, để tính KL divergence)
# Thường là bản sao của model gốc trước PPO
ppo_model_ref = AutoModelForCausalLMWithValueHead.from_pretrained(
    base_llm_model_name,
    # quantization_config=bnb_config, # Bỏ comment nếu dùng 4-bit
    pad_token_id=ppo_tokenizer.pad_token_id,
    device_map={"": device} if device == "cpu" else "auto", # Chỉ định device map cho CPU hoặc để accelerate xử lý GPU
    # torch_dtype=torch.bfloat16
)
ppo_model_ref.eval() # Chuyển model tham chiếu sang chế độ đánh giá

print("Đã load các model cho PPO.")

# 2.2. Chuẩn bị Dữ liệu Prompt cho PPO
# Các câu prompt đơn giản để model hoàn thành theo hướng tích cực
prompts = [
    "The movie plot was",
    "I thought the acting was",
    "This film is definitely",
    "Overall, the cinema experience felt",
    "The director managed to make it",
]

# *** FIX: Kế thừa từ torch.utils.data.Dataset ***
class PromptDataset(TorchDataset): # Kế thừa từ PyTorch Dataset
    def __init__(self, prompts, tokenizer):
        self.prompts = prompts
        self.tokenizer = tokenizer
        # Tokenize tất cả prompt một lần và lưu trữ tensor
        # Lưu ý: không squeeze() ở đây, collator sẽ xử lý padding
        self.tokenized_prompts = [
            self.tokenizer(prompt, return_tensors="pt") for prompt in self.prompts
        ]

    def __len__(self):
        return len(self.prompts)

    def __getitem__(self, idx):
        # Trả về tensor đã được tokenize và query text gốc
        tokenized = self.tokenized_prompts[idx]
        # Lấy input_ids và attention_mask (vẫn là 2D tensor [1, seq_len] ở đây)
        input_ids = tokenized.input_ids.squeeze(0) # Squeeze về 1D
        attention_mask = tokenized.attention_mask.squeeze(0) # Squeeze về 1D
        return {"input_ids": input_ids, "attention_mask": attention_mask, "query": self.prompts[idx]}

# Khởi tạo dataset đã sửa
ppo_dataset = PromptDataset(prompts, ppo_tokenizer)

def collator(data):
    # Ghép các tensor lại thành batch
    # Input bây giờ là list các tensor 1D từ __getitem__
    input_ids_list = [item['input_ids'] for item in data]
    # attention_mask_list = [item['attention_mask'] for item in data] # Không cần mask cho generate, nhưng có thể cần cho step
    queries = [item['query'] for item in data] # Giữ lại dạng text để debug/sử dụng

    # Pad các tensors trong batch để tạo tensor 2D
    padded_input_ids = torch.nn.utils.rnn.pad_sequence(input_ids_list, batch_first=True, padding_value=ppo_tokenizer.pad_token_id)
    # padded_attention_mask = torch.nn.utils.rnn.pad_sequence(attention_mask_list, batch_first=True, padding_value=0) # Pad attention mask nếu cần

    # Trả về batch dưới dạng dictionary, bao gồm cả query text
    # PPOTrainer sẽ tự động chuyển batch lên device phù hợp
    return {
        "input_ids": padded_input_ids,
        # "attention_mask": padded_attention_mask, # Bỏ comment nếu PPOTrainer.step yêu cầu
        "query": queries
    }

print(f"Đã tạo dataset PPO với {len(ppo_dataset)} prompts.")

# 2.3. Cấu hình PPO
ppo_config = PPOConfig(
    model_name=base_llm_model_name,
    learning_rate=1.4e-5, # Thử nghiệm với learning rate này
    batch_size=len(prompts),       # Xử lý tất cả prompts trong 1 batch PPO
    mini_batch_size=1,     # Update PPO trên từng cặp prompt/response
    gradient_accumulation_steps=1, # Tích lũy gradient qua 1 bước (không tích lũy)
    ppo_epochs=4,          # Số vòng lặp tối ưu trên mỗi batch
    log_with=None,         # Tắt logging bên ngoài
    remove_unused_columns=False, # Giữ lại các cột thừa như "query"
    kl_penalty="kl",       # Sử dụng KL penalty
    target_kl=0.1,         # KL target
    adap_kl_ctrl=True,     # Tự động điều chỉnh hệ số KL
    seed=42,               # Để có thể tái lập kết quả
    use_score_scaling=False, # Tắt scale điểm số
    use_score_norm=False,    # Tắt chuẩn hóa điểm số
    score_clip=None,         # Không cắt điểm số
)

# --- Hàm tính Reward từ Sentiment Classifier ---
def get_sentiment_reward(texts, reward_tokenizer, reward_model, target_label=1, sentiment_scale=5.0, reward_device='cpu'):
    """
    Tính reward dựa trên điểm số sentiment.
    target_label = 1 nghĩa là muốn kết quả POSITIVE.
    sentiment_scale: Hệ số khuếch đại điểm sentiment thành reward.
    reward_device: Device để lưu trữ các tensor reward cuối cùng ('cpu' được khuyến nghị).
    """
    rewards = []
    with torch.no_grad(): # Tắt tính toán gradient để inference
        for text in texts:
            try:
                # Tokenize cho RM
                inputs = reward_tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
                # Chuyển inputs lên device của reward model
                inputs = {k: v.to(reward_model.device) for k, v in inputs.items()}

                # Lấy logits từ RM
                logits = reward_model(**inputs).logits[0] # Lấy logits của mẫu đầu tiên (và duy nhất)

                # Chuyển logits thành xác suất (softmax)
                probs = torch.softmax(logits, dim=-1)

                # Lấy điểm số cho nhãn mục tiêu (ví dụ: positive = 1)
                score = probs[target_label]

                # Scale điểm số thành reward (có thể điều chỉnh cách scale)
                # Ở đây, dùng score * scale làm reward
                reward = score * sentiment_scale
                # Đảm bảo reward là tensor trên device chỉ định
                rewards.append(torch.tensor(reward.item(), device=reward_device))
            except Exception as e:
                print(f"Lỗi khi tính reward cho text: '{text}'. Lỗi: {e}")
                # Gán reward mặc định (ví dụ: 0) nếu có lỗi
                rewards.append(torch.tensor(0.0, device=reward_device))


    # Trả về list các tensor reward
    return rewards


# 2.4. Khởi tạo PPOTrainer
# PPOTrainer mong đợi dataset yield ra các dictionary
# trong đó 'input_ids' và 'attention_mask' là key cho các prompt đã được tokenize
ppo_trainer = PPOTrainer(
    config=ppo_config,
    model=ppo_model,
    ref_model=ppo_model_ref,
    tokenizer=ppo_tokenizer,
    dataset=ppo_dataset,      # Truyền PromptDataset đã khởi tạo
    data_collator=collator, # Truyền hàm collator
)

# 2.5. Vòng lặp Huấn luyện PPO
generation_kwargs = {
    "min_length": -1, # Cho phép dừng sinh sớm
    "top_k": 0.0,     # Tắt top-k sampling
    "top_p": 1.0,     # Tắt nucleus sampling
    "do_sample": True,# Bật sampling
    "pad_token_id": ppo_tokenizer.pad_token_id,
    "eos_token_id": ppo_tokenizer.eos_token_id if ppo_tokenizer.eos_token_id is not None else -1, # Đặt EOS token nếu có
    "max_new_tokens": 48, # Sinh tối đa 48 token mới
}

print("Bắt đầu huấn luyện PPO...")
num_ppo_iterations = 10 # Chạy PPO trong 10 vòng lặp (mỗi vòng xử lý batch_size prompts)

for iteration in range(num_ppo_iterations):
    print(f"\n--- PPO Iteration: {iteration+1}/{num_ppo_iterations} ---")

    # Dataloader được PPOTrainer tự động tạo từ dataset và collator
    batch = next(iter(ppo_trainer.dataloader))

    # Lấy batch tensor 2D từ dataloader (đã được collate và pad)
    # PPOTrainer sẽ tự động chuyển lên device phù hợp
    prompt_tensors_batch = batch["input_ids"] # Tensor 2D (batch_size, padded_seq_len)
    queries = batch["query"] # List các query text

    # *** FIX: Lặp qua batch và truyền tensor 1D vào generate ***
    response_tensors_list = [] # List để lưu các response tensor 1D
    prompt_tensors_list = []   # List để lưu các prompt tensor 1D (cho hàm step)

    for i in range(prompt_tensors_batch.size(0)):
        # Lấy tensor prompt 1D từ batch 2D
        # Cần loại bỏ padding trước khi truyền vào generate nếu tokenizer không tự xử lý
        # Hoặc đảm bảo generate xử lý được padding (thường là vậy)
        prompt_tensor_1d = prompt_tensors_batch[i] # Tensor 1D (padded_seq_len)

        # Lưu lại tensor 1D này để dùng cho hàm step
        prompt_tensors_list.append(prompt_tensor_1d)

        try:
            # Gọi generate với tensor 1D
            # PPOTrainer.generate sẽ đặt tensor lên đúng device
            response_tensor_1d = ppo_trainer.generate(
                prompt_tensor_1d, # Truyền tensor 1D
                return_prompt=False, # Chỉ lấy phần response
                length_sampler=LengthSampler(generation_kwargs["max_new_tokens"] // 2, generation_kwargs["max_new_tokens"]),
                **generation_kwargs,
            )
            # response_tensor_1d thường là tensor 1D chứa các token ID của response
            # Loại bỏ chiều batch nếu generate trả về (1, seq_len) - kiểm tra output của generate
            if response_tensor_1d.dim() > 1:
                 response_tensor_1d = response_tensor_1d.squeeze(0)

            response_tensors_list.append(response_tensor_1d)

        except Exception as e:
            print(f"Lỗi khi generate cho prompt {i}: {queries[i]}. Lỗi: {e}")
            # Xử lý lỗi: có thể thêm tensor rỗng hoặc bỏ qua mẫu này
            # Thêm tensor rỗng để giữ đúng số lượng cho hàm step
            response_tensors_list.append(torch.tensor([], dtype=torch.long, device=ppo_trainer.accelerator.device))


    # Kiểm tra xem có response nào được sinh ra không
    if not response_tensors_list or all(t.numel() == 0 for t in response_tensors_list):
        print("Không có response nào được sinh ra trong iteration này. Bỏ qua bước PPO.")
        continue # Bỏ qua iteration này nếu không có response hợp lệ

    # Decode responses thành text (chỉ decode những response không rỗng)
    valid_indices = [idx for idx, t in enumerate(response_tensors_list) if t.numel() > 0]
    valid_responses_text = ppo_tokenizer.batch_decode([response_tensors_list[i] for i in valid_indices], skip_special_tokens=True)
    responses_text = [""] * len(queries) # Khởi tạo list rỗng
    for i, text in zip(valid_indices, valid_responses_text):
        responses_text[i] = text # Điền vào vị trí đúng

    # Ghép prompt và response text để tính reward
    full_texts = [q + r for q, r in zip(queries, responses_text)]

    # 2. Tính Reward từ Reward Model (Sentiment Classifier)
    # Đảm bảo rewards được trả về là tensor trên CPU
    rewards = get_sentiment_reward(full_texts, reward_tokenizer, reward_model, target_label=1, sentiment_scale=5.0, reward_device='cpu')

    # 3. Thực hiện bước tối ưu PPO
    # Cần cung cấp list các tensor 1D cho prompts, responses, và rewards
    # Đảm bảo các list có cùng độ dài và tương ứng
    # Chỉ thực hiện step nếu có response hợp lệ
    if valid_indices:
        # Lọc ra các prompt và response tương ứng với các response hợp lệ
        valid_prompt_tensors = [prompt_tensors_list[i] for i in valid_indices]
        valid_response_tensors = [response_tensors_list[i] for i in valid_indices]
        valid_rewards = [rewards[i] for i in valid_indices] # Giả sử rewards tính cho cả text rỗng là 0

        # Kiểm tra lại độ dài trước khi gọi step
        if len(valid_prompt_tensors) == len(valid_response_tensors) == len(valid_rewards):
             try:
                stats = ppo_trainer.step(valid_prompt_tensors, valid_response_tensors, valid_rewards)
             except Exception as e:
                 print(f"Lỗi trong ppo_trainer.step: {e}")
                 print(f"  len(prompts): {len(valid_prompt_tensors)}, len(responses): {len(valid_response_tensors)}, len(rewards): {len(valid_rewards)}")
                 # Có thể in ra shape của các tensor để debug thêm
                 continue # Bỏ qua phần log nếu step lỗi
        else:
            print("Độ dài không khớp giữa prompts, responses và rewards sau khi lọc. Bỏ qua bước PPO.")
            stats = {} # Gán stats rỗng để tránh lỗi log bên dưới
    else:
        print("Không có response hợp lệ nào để thực hiện bước PPO.")
        stats = {} # Gán stats rỗng

    # Log thông tin
    print(f"Iteration {iteration+1} completed.")
    try:
        # Tính mean reward từ các reward hợp lệ
        if valid_indices:
            mean_reward = torch.stack([rewards[i] for i in valid_indices]).mean().item()
            print(f"  Mean Reward this batch (valid responses): {mean_reward:.4f}")
        else:
            print("  Mean Reward this batch: N/A (no valid responses)")

        # Lấy KL divergence một cách an toàn từ dictionary stats (nếu stats không rỗng)
        if stats:
             mean_kl = stats.get('objective/kl', float('nan')) # Dùng NaN nếu không tìm thấy key
             print(f"  Mean KL Div this batch: {mean_kl:.4f}" if isinstance(mean_kl, (float, int)) and not np.isnan(mean_kl) else 'N/A')
        else:
             print("  Mean KL Div this batch: N/A (step skipped or failed)")

    except Exception as e:
        print(f"  Could not calculate batch stats: {e}")
        # print(f"  Rewards: {rewards}") # In rewards để debug
        # print(f"  Stats: {stats}") # In stats dict để debug

    # In ví dụ prompt, response, và reward (chỉ in nếu có response hợp lệ)
    if valid_indices:
        example_idx = valid_indices[0] # Lấy index của ví dụ hợp lệ đầu tiên
        print("  Example:")
        print(f"    Query: {queries[example_idx]}")
        print(f"    Response: {responses_text[example_idx]}")
        # print(f"    Full Text: {full_texts[example_idx]}") # Có thể bỏ comment nếu muốn xem full text
        print(f"    Reward: {rewards[example_idx].item():.4f}")


print("\nHuấn luyện PPO hoàn tất.")

# 2.6. Lưu Model PPO đã tinh chỉnh
print(f"Lưu model PPO đã tinh chỉnh tại: {output_dir_ppo}")
# Lưu model cơ sở (không có value head) để dễ dàng load bằng pipeline
# Value head thường không được lưu cho inference
ppo_trainer.save_pretrained(output_dir_ppo)

# --- Bước 3: Kiểm tra Kết quả ---
print("\n--- Bắt đầu Bước 3: Kiểm tra Kết quả ---")

# Load model đã fine-tune bằng PPO sử dụng pipeline chuẩn
# Tốt nhất là load kiến trúc model cơ sở rồi load trọng số PPO đã lưu
try:
    # Load model được lưu bởi PPOTrainer (nên là model cơ sở)
    final_model = AutoModelForCausalLM.from_pretrained(output_dir_ppo)
    final_tokenizer = AutoTokenizer.from_pretrained(output_dir_ppo)
    final_model.to(device) # Chuyển model cuối cùng lên device phù hợp

    # Tạo pipeline text generation
    # device=0 tương ứng với CUDA device đầu tiên nếu có, -1 tương ứng với CPU
    pipe = pipeline("text-generation", model=final_model, tokenizer=final_tokenizer, device=0 if device=="cuda" else -1)

    print("\nSinh thử nghiệm với model PPO:")
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        # Sinh nhiều chuỗi để xem sự đa dạng
        outputs = pipe(
            prompt,
            max_new_tokens=50,
            num_return_sequences=2,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            pad_token_id=final_tokenizer.eos_token_id if final_tokenizer.eos_token_id is not None else ppo_tokenizer.pad_token_id # Sử dụng EOS hoặc PAD token
        )
        for i, out in enumerate(outputs):
            generated_text = out['generated_text']
            print(f"  Output {i+1}: {generated_text}")
            # Đánh giá sentiment của văn bản được sinh ra bằng RM
            # Đảm bảo việc tính reward sử dụng đúng reward model và tokenizer
            # Chuyển reward tensor về CPU để lấy item()
            reward_tensor = get_sentiment_reward([generated_text], reward_tokenizer, reward_model, target_label=1, reward_device='cpu')[0]
            print(f"    Sentiment Reward (Positive target): {reward_tensor.item():.4f}")

except Exception as e:
    print(f"\nLỗi khi tạo pipeline hoặc sinh thử nghiệm: {e}")
    print("Hãy kiểm tra xem model đã được lưu đúng cách chưa.")
    print(f"Model đã được lưu tại: {output_dir_ppo}")

print("\nHoàn thành!")

Sử dụng thiết bị: cuda

--- Bắt đầu Bước 1: Huấn luyện Reward Model (Sentiment Classifier) ---


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Iteration 4 completed.
  Mean Reward this batch (valid responses): 0.0901
  Mean KL Div this batch: 1.2318
  Example:
    Query: The director managed to make it
    Response:  into a deeply flawed program with artificially stylish Simpsons a hot flyer. See below for what matters, and if he won't, it may prove to be
    Reward: 0.0694

--- PPO Iteration: 5/10 ---
Iteration 5 completed.
  Mean Reward this batch (valid responses): 0.0881
  Mean KL Div this batch: 2.2149
  Example:
    Query: The director managed to make it
    Response:  clear mostly by sending new 25-minute ads out through a YouTube channel that was denied ever to produce these ads. Also, it also
    Reward: 0.1303

--- PPO Iteration: 6/10 ---
Iteration 6 completed.
  Mean Reward this batch (valid responses): 0.0961
  Mean KL Div this batch: 3.5749
  Example:
    Query: Overall, the cinema experience felt
    Response:  real and did the same. While in Illogany, visual Magic presented both a possible form of expectation a

/usr/local/lib/python3.11/dist-packages/trl/trainer/ppo_trainer.py:1431: UserWarning: Cannot retrieve user information assuming you are running in offline mode.
  warnings.warn("Cannot retrieve user information assuming you are running in offline mode.")
Some weights of the model checkpoint at ./ppo_positive_generator were not used when initializing GPT2LMHeadModel: ['v_head.summary.bias', 'v_head.summary.weight']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



--- Bắt đầu Bước 3: Kiểm tra Kết quả ---


Device set to use cuda:0



Sinh thử nghiệm với model PPO:

Prompt: The movie plot was
  Output 1: The movie plot was as follows:
A character who had been raised by his grandparents in Poland was a Jewish man from Warsaw, Poland (his parents were Polish, and he lived in Poland). He was educated at a city high school, where he spent summers with Polish
    Sentiment Reward (Positive target): 0.0900
  Output 2: The movie plot was created by a fictional writer and writer named Richard Branson as a fictional character.


The film in question was a pre-scripted version of the story of the movie's opening act which was played as the young, lonely and mysterious Richard
    Sentiment Reward (Positive target): 0.0773

Prompt: I thought the acting was
  Output 1: I thought the acting was done because he is looking for something to do with the characters of his TV show. He was like, "Oh, well, it's like he's on a boat. You think he is walking around here, looking for something to do with the
    Sentiment Reward (Positive

In [8]:
import trl
print(trl.__version__)

0.11.3
